# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

import json
import geopandas as gpd

# Import API key
from api_keys import geoapify_key

In [3]:
import holoviews as hv

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,arauco,-37.2463,-73.3175,287.47,86,49,1.89,CL,1713216124
1,1,suez,29.9737,32.5263,293.23,58,0,4.06,EG,1713216141
2,2,hard bargain,26.3167,-77.5667,295.83,64,0,5.84,BS,1713216143
3,3,new norfolk,-42.7826,147.0587,284.10,84,72,0.98,AU,1713216144
4,4,whitehorse,60.7161,-135.0538,277.09,34,20,2.57,CA,1713216145


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
# %%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 800,
    frame_height = 600,
    scale = 2,
    size = 'Humidity',
    color = 'City',
    alpha = 0.5
)

# Display the map
city_map

c:\Users\ancol\.vscode\New folder\envs\dev\lib\site-packages\holoviews\core\util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.5, use 'transform_reference' instead.
  value = param_value_if_widget(value)
c:\Users\ancol\.vscode\New folder\envs\dev\lib\site-packages\holoviews\core\util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.5, use 'transform_reference' instead.
  value = param_value_if_widget(value)


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
#Convert Max Temp from Kelvin to Celsius
city_data_df['Max Temp']=city_data_df['Max Temp']-273.15

#Filter max temp for <27 and >22
city_data_df = city_data_df[(city_data_df['Max Temp'] > 22) & (city_data_df['Max Temp']<27) & (city_data_df['Wind Speed']<4.5) & (city_data_df['Cloudiness']==0)]

# filtered_df = city_data_df[(city_data_df['Max Temp'] > 22) & (city_data_df['Max Temp'] < 27)]
# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
22,22,saipan,15.1355,145.7010,26.39,83,0,2.06,MP,1713216000
261,261,karratha,-20.7377,116.8463,26.17,71,0,4.41,AU,1713216507
369,369,griffith,41.5284,-87.4237,25.17,34,0,3.09,US,1713216656
400,400,vilhena,-12.7406,-60.1458,26.12,89,0,1.54,BR,1713216703
445,445,taoudenni,22.6783,-3.9836,26.81,17,0,3.49,ML,1713216769
513,513,kota belud,6.3510,116.4305,25.43,79,0,1.54,MY,1713216593
542,542,darwin,-12.4611,130.8418,24.99,83,0,3.09,AU,1713216912


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.iloc[:,[1,8,2,3,5]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

C:\Users\ancol\AppData\Local\Temp\ipykernel_3452\1185728973.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


,City,Country,Lat,Lng,Humidity,Hotel Name
22,saipan,MP,15.1355,145.7010,83,
261,karratha,AU,-20.7377,116.8463,71,
369,griffith,US,41.5284,-87.4237,34,
400,vilhena,BR,-12.7406,-60.1458,89,
445,taoudenni,ML,22.6783,-3.9836,17,
513,kota belud,MY,6.3510,116.4305,79,
542,darwin,AU,-12.4611,130.8418,83,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories':'accommodation.hotel',
    'apiKey': geoapify_key,
    'limit':1,
    'radius':radius
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saipan - nearest hotel: Chalan Kanoa Beach Hotel
karratha - nearest hotel: Karratha International Hotel
griffith - nearest hotel: Quality Inn & Suites Hammond
vilhena - nearest hotel: No hotel found
taoudenni - nearest hotel: No hotel found
kota belud - nearest hotel: U Hotel
darwin - nearest hotel: Mantra Pantanas Darwin


,City,Country,Lat,Lng,Humidity,Hotel Name
22,saipan,MP,15.1355,145.7010,83,Chalan Kanoa Beach Hotel
261,karratha,AU,-20.7377,116.8463,71,Karratha International Hotel
369,griffith,US,41.5284,-87.4237,34,Quality Inn & Suites Hammond
400,vilhena,BR,-12.7406,-60.1458,89,No hotel found
445,taoudenni,ML,22.6783,-3.9836,17,No hotel found
513,kota belud,MY,6.3510,116.4305,79,U Hotel
542,darwin,AU,-12.4611,130.8418,83,Mantra Pantanas Darwin


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points('Lng','Lat',geo=True, tiles = 'OSM', color='City',size='Humidity',hover_cols='all')

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,index,Country,Hotel Name)